# 各種構造化テキストを読み込んでPythonオブジェクトを各種データベースに保存・読み出し

In [58]:
from facility import facility
import ipywidgets as widgets
from IPython.display import display, clear_output

In [59]:
input_format = widgets.RadioButtons(
    options=['定型文字列','XML', 'JSON',  'RDB', 'MongoDB'],
    values =['TEXT','XML','JSON', 'RDB',  'MongoDB'],
    description='入力形式：',
    disabled=False
)
table_name = widgets.Text(
    value='',
    placeholder='RDBのテーブル名にあたるオブジェクト名',
    description='オブジェクト名：',
    disabled=False
)
facility_str = widgets.Textarea(
    value='',
    placeholder='各種形式のオブジェクト文字列を貼り付け',
    description='施設情報：',
    disabled=False
)
output_format = widgets.RadioButtons(
    options=['XML', 'JSON', 'MongoDB', 'RDB','PythonのDict型の文字列'],
    values =['XML','JSON','MongoDB','RDB','Python'],
    description='出力形式：',
    disabled=False
)
button = widgets.Button(description="送信")

display(input_format,  table_name, facility_str, output_format,button)

RadioButtons(description='入力形式：', options=('定型文字列', 'XML', 'JSON', 'RDB', 'MongoDB'), value='定型文字列')

Text(value='', description='オブジェクト名：', placeholder='RDBのテーブル名にあたるオブジェクト名')

Textarea(value='', description='施設情報：', placeholder='各種形式のオブジェクト文字列を貼り付け')

RadioButtons(description='出力形式：', options=('XML', 'JSON', 'MongoDB', 'RDB', 'PythonのDict型の文字列'), value='XML')

Button(description='送信', style=ButtonStyle())

SELECT * FROM facility;
[2, 'name', '茅ケ崎市立小出小学校']
[2, 'category', '公立小学校']
[2, 'e_category', '地区防災拠点']
[2, 'lodation', '神奈川県茅ケ崎市芹沢994']
[2, 'contact', '0467-51-8814']
[2, 'latitude', '35.371811']
[2, 'longitude', '139.4218083']
[2, 'st_type', '防災備蓄倉庫（コンテナ）']
[4, 'name', '茅ケ崎市立小出小学校']
[4, 'category', '公立小学校']
[4, 'e_category', '地区防災拠点']
[4, 'lodation', '神奈川県茅ケ崎市芹沢994']
[4, 'contact', '0467-51-8814']
[4, 'latitude', '35.371811']
[4, 'longitude', '139.4218083']
[4, 'st_type', '防災備蓄倉庫（コンテナ）']
[6, 'name', '茅ケ崎市立小出小学校']
[6, 'category', '公立小学校']
[6, 'e_category', '地区防災拠点']
[6, 'lodation', '神奈川県茅ケ崎市芹沢994']
[6, 'contact', '0467-51-8814']
[6, 'latitude', '35.371811']
[6, 'longitude', '139.4218083']
[6, 'st_type', '防災備蓄倉庫（コンテナ）']
list_size=4
output_formatXML


In [60]:
import re
def readFormatedText(input_str):
    
    input_lines = re.split('\n',input_str)#改行文字(\n)で文字列を分割

    isID = 0
    new_obj_dict = {}
    for input_line in input_lines:
        elems = []
        elems = re.split("\s+",input_line)#空白文字(\s)で文字列を分割
        if len(elems) < 3 :#入力文字列内の改行のみの行などで要素が足りないことを防ぐ
            continue
        if len(elems[0]) != 0:#先頭のセルが空ではない（idの値が入っている）
            if isID == 0:#最初のidの値であるかどうか
                isID = 1
            else:#2つ目以降のid値が入った行である場合
                new_obj = facility(init_values_dic=new_obj_dict)#新たなfacility型インスタンスを作成
                facility_list.append(new_obj) #結果用配列（リスト）に新たな結果を追加
                new_obj_dict = {}#new_obj_dictを新たな空のdict型にする
                isID += 1
            new_obj_dict[elems[2]]=elems[3]#フィールド変数名と対応する値を読み取る
        else:
            new_obj_dict[elems[2]]=elems[3]#フィールド変数名と対応する値を読み取る

    new_obj = facility(init_values_dic=new_obj_dict)#新たなfacility型インスタンスを作成（最後の分）
    facility_list.append(new_obj)#結果用配列（リスト）に新たな結果を追加（最後の分）


In [61]:
# xmldtodictがimportできない場合は，下記のコマンドをコメントを外して一回Runし，ipynbを再起動する
#!pip install xmltodict --user
import xmltodict
def readXML(input_str):

    tmp_dict = xmltodict.parse(input_str)
    for class_name in tmp_dict["facility_list"]:
        if type(tmp_dict["facility_list"][class_name]) is not list:
            facility_list.append(facility(tmp_dict["facility_list"][class_name]))
        else:
            for obj_dict in tmp_dict["facility_list"][class_name]:
                facility_list.append(facility(obj_dict))


In [62]:
def readJSON(input_str):

    tmp_list = json.loads(input_str)
    for t_dict in tmp_list:
        new_dict = {}
        for t_key in t_dict.keys():
           new_key = re.search("_(.+?)__(.+)",t_key).group(2)
           new_dict[new_key]=t_dict[t_key]
        facility_list.append(facility(new_dict))


In [63]:
from pymongo import MongoClient
def readMongoDB():
    client = MongoClient('localhost', 27017)#データベースサーバへの接続（今回は演習向けにパスワード無し）
    db = client.各自のユーザ名 #「各自のユーザ名」のところは，b9p31000のように変更する
    collection = db.facility #facilityはコレクション名（テーブルに相当）
    
    for facility_obj in collection.find():
        #print(type(facility_obj)) #typeではdictと表示される
        new_dict={}
        for t_key in facility_obj.keys():
            if re.match("_(.+?)__(.+)",t_key):
                new_key = re.search("_(.+?)__(.+)",t_key).group(2)
                new_dict[new_key]=facility_obj[t_key]
                
        facility_list.append(facility(new_dict))
    

In [64]:
import psycopg2
import psycopg2.extras

def readRDB():
    
    db_host = "localhost"
    db_name = "b9p31049"
    db_user = "b9p31049"
    db_password = "B9P31049"
    
    conn_str = "host=" +db_host+" dbname=" +db_name+" user="+db_user+" password=" +db_password
    
    connection = psycopg2.connect(conn_str) #データベースに接続
    cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor) #dict型でデータを読み書きすることを設定
    
    #データベースの指定されたテーブル名をクラス名として読み取る
    sql = "SELECT * FROM "+table_name.value+";"
    print(sql)
    cursor.execute(sql)
    
    facility_dict = {}
    prev_id = -1
    for row in cursor:
        print(row)
        if prev_id == 0:
            prev_id = row["id"]
        if row["id"] == prev_id:
            facility_dict[row["var_name"]]=row["var_value"]
        else:
            facility_list.append(facility(facility_dict))
            facility_dict = {}
            facility_dict[row["var_name"]]=row["var_value"]
            prev_id = row["id"]

    facility_list.append(facility(facility_dict))
    
    cursor.close()
    connection.close()

In [65]:
def toXML():
        facility_list_str = '<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n'
        facility_list_str += '<facility_list>\n'

        for elem_obj in facility_list:
            class_name = elem_obj.__class__.__name__
            elem_obj_xml = '  <'+class_name+'>\n'
            elem_obj_dict = elem_obj.toDict()
            for key in elem_obj_dict.keys():
                name_space = '_'+class_name+'__'
                val_name = key.replace(name_space,"")
                elem_obj_xml += '    <'+val_name+'>'
                elem_obj_xml += elem_obj_dict[key]
                elem_obj_xml += '</'+val_name+'>\n'
            elem_obj_xml += '  </'+class_name+'>\n'
            facility_list_str += elem_obj_xml

        facility_list_str += '</facility_list>\n'
        
        return facility_list_str

In [66]:
from pymongo import MongoClient
def toMongoDB():
    client = MongoClient('localhost', 27017)#データベースサーバへの接続（今回は演習向けにパスワード無し）
    db = client.各自のユーザ名 #「各自のユーザ名」のところは，b9p31000のように変更する
    collection = db.facility #facilityはコレクション名（テーブルに相当）

    #Pythonのオブジェクトの配列をMongoDBの各自のfacilityコレクションに保存
    for facility in facility_list:
        collection.insert_one(facility.toDict())
    
    #追加後のfacilityコレクションの内容を表示
    result_str = ""
    for facility_obj in collection.find():
        result_str += str(facility_obj)
    
    return result_str

In [67]:
import psycopg2
import psycopg2.extras

def toRDB():

    db_host = "localhost"
    db_name = "b9p31049"#学籍番号のユーザ名
    db_user = "b9p31049"#学籍番号のユーザ名
    db_password = "B9P31049"#学籍番号のユーザ名のBとPを大文字にした文字列
    
    result_str = ""

    conn_str = "host="+db_host+" dbname="+db_name+" user="+db_user+" password="+db_password
    print(conn_str)
    connection = psycopg2.connect(conn_str) #データベースに接続
    cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor) #dict型でデータを読み書きすることを設定

    #facility_list内の各要素をデータベースのクラス名のテーブルに記録する
    num = 0
    id = 0
    for elem_obj in facility_list:
        # INSERT文を作る
        class_name = elem_obj.__class__.__name__
        if id == 0:
            select_sql = "SELECT MAX(id) FROM "+class_name+";"
            print(select_sql)
            cursor.execute(select_sql)
            for row in cursor:
                id = row["max"]
        if id == None:
            id = 0
        id = id + 1
        elem_obj_dict={}
        elem_obj_dict = elem_obj.toDict()
        for key in elem_obj_dict.keys():
            name_space = '_'+class_name+'__'
            var_name = key.replace(name_space,"")
            var_value = elem_obj_dict[key]

            insert_sql = "INSERT INTO "+class_name+" VALUES ("
            insert_sql += str(id)+", "
            insert_sql += "'"+var_name+"',"
            insert_sql += "'"+var_value+"'"
            insert_sql += ");"
            print(insert_sql)
            cursor.execute(insert_sql) #INSERT文を実行する
        connection.commit()#１つのオブジェクトの処理が終わるごとにコミットを行う（psycopg2は特に設定しないと内部でトランザクションを作る）
        num += 1

    cursor.close()
    connection.close()
    
    result_str = str(num)+"件のデータをデータベースに登録しました。"

    return result_str

In [68]:
import json
def on_button_clicked(b):#「送信」ボタンをクリックしたときに実行される処理
    
    #新しいfacility型オブジェクトの配列を作成
    facility_list.clear()
    if input_format.value == "XML":
        readXML(facility_str.value)
    elif input_format.value == "JSON":
        readJSON(facility_str.value)
    elif input_format.value == "RDB":
        readRDB()
    elif input_format.value == "MongoDB":
        readMongoDB()
    else:
        readFormatedText(facility_str.value)
    
    facility_list_str = ""
    print("list_size="+str(len(facility_list)))
    print("output_format"+str(output_format.value))
    if output_format.value == "XML":
        facility_list_str = toXML()
    elif output_format.value == "JSON":
        facility_list_str = \
          json.dumps(facility_list,default=vars, ensure_ascii=False)
    elif output_format.value == "MongoDB":
        facility_list_str = toMongoDB()
    elif output_format.value == "RDB":
        facility_list_str = toRDB()    
    else:
        for obj in facility_list:
            facility_list_str = facility_list_str + str(obj.toDict())
        
    with output:
        clear_output()
        print(facility_list_str)


In [69]:
button.on_click(on_button_clicked)

facility_list=list()

output = widgets.Output()

display(output)

Output()